In [66]:
from google.colab import drive 
drive.mount('/content/drive')

#IMPORTS

import os
from google.colab import files
import matplotlib.pyplot as plt
import math
import numpy as np
from PIL import Image
import cv2 as cv
import random
import tensorflow as tf
from tensorflow import keras

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
data_dir = '/content/drive/MyDrive/Bismuth Data/'

In [68]:
for file in os.scandir(data_dir):
  print(file.path)

/content/drive/MyDrive/Bismuth Data/2021-02-08
/content/drive/MyDrive/Bismuth Data/2021-02-09
/content/drive/MyDrive/Bismuth Data/2021-02-11


In [69]:
#source: https://www.codespeedy.com/how-to-iterate-over-files-in-a-given-directory-in-python/
def process_data(dims):
  data_array = []
  for subdirectories, directories, files in os.walk(data_dir):
    for file_name in files:
      file_loc = subdirectories + os.path.sep + file_name
      #print(file_name)
      photo = Image.open(file_loc)#"filename.tif")
      
      #adding some black pixels to the top and bottom so that the image's aspect ratio doesn't change when resizing
      buffer_array = np.zeros((74, 1148))
      photo_arr = np.concatenate((buffer_array,np.array(photo),buffer_array),axis=0)/255
      
      resized_photo = cv.resize(photo_arr,dims)
      data_array.append([resized_photo,file_name])

      """
      #Comparing images before/after processing
      fig = plt.figure(figsize=(10, 10))
      fig.add_subplot(1, 2, 1)
      plt.imshow(photo_arr,cmap='gray')
      plt.title("Original Image")
      fig.add_subplot(1, 2, 2)
      plt.imshow(resized_photo,cmap='gray)
      plt.title("Resized Image")
      plt.show()
      """
  print(len(data_array))
  return data_array

859


In [ ]:
dims = (150,150)

data_array = process_data(dims) 

In [ ]:
photo = np.zeros((74, 1148))
